<a href="https://colab.research.google.com/github/obeabi/SdsOctober2020Challenge/blob/main/SelectiveEncoding_Main.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Written by Abiola Obembe
# SDS October Challenge #2 - Used Car Prices
## 2021-01-09

## Selective Feature engineering of categorical columns

This month you have been hired by a new car trader company to help sell its used cars. You will be analyzing used cars from multiple manufacturers and of different models. Your job is to help the car trader company determine the price of its used cars.

## Step 1: Data Preprocessing 

In [43]:
# Download packages
!pip install category_encoders

In [44]:
# Importing the libraries
import pickle
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import seaborn as sns
from scipy import stats
from scipy.stats import f_oneway
from scipy.stats import  zscore
import category_encoders as ce
from sklearn.preprocessing import StandardScaler 
from sklearn.linear_model  import Ridge,Lasso,RidgeCV, LassoCV, ElasticNet, ElasticNetCV, LinearRegression
from sklearn.model_selection import train_test_split

%matplotlib inline
plt.rcParams['figure.figsize'] = (8.0, 6.0) # set default size of plots
plt.rcParams['image.interpolation'] = 'nearest'
plt.rcParams['image.cmap'] = 'gray'

print('Libraries installed successfully!')

np.random.seed(0)

Libraries installed successfully!


In [45]:
# Suppressing warnings
import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)

In [46]:
# Importing the training dataset

df_train = pd.read_csv('Train.csv', engine= 'python', encoding= 'latin1')
df_train.drop('Unnamed: 0', axis = 1, inplace = True)

df_train.head()

,manufacturer_name,model_name,transmission,color,odometer_value,year_produced,engine_fuel,engine_has_gas,engine_type,engine_capacity,body_type,has_warranty,state,drivetrain,feature_0,feature_1,feature_2,feature_3,feature_4,feature_5,feature_6,feature_7,feature_8,feature_9,duration_listed,price_usd
0,Ford,Fusion,mechanical,blue,245000,2006,gasoline,0,gasoline,1.6,hatchback,0,owned,front,1,0,0,0,0,0,0,0,0,0,7,4250.0
1,Dodge,Caravan,automatic,silver,265542,2002,gasoline,0,gasoline,3.3,minivan,0,owned,front,0,1,0,0,0,0,0,0,0,0,133,4000.0
2,Ford,Galaxy,mechanical,blue,168000,2009,diesel,0,diesel,1.8,minivan,0,owned,front,0,0,0,1,0,0,1,1,1,1,0,10900.0
3,Mazda,6,mechanical,other,225522,2008,gasoline,0,gasoline,1.8,universal,0,owned,front,0,1,1,0,0,1,0,0,1,1,20,6999.0
4,Audi,80,mechanical,black,370000,1991,gasoline,0,gasoline,1.8,sedan,0,owned,front,0,0,0,0,0,0,0,0,0,1,160,1600.0


In [47]:
# priniting the details about the train dataset

print("\nEDA on Training Set\n")
print("#"*30)

print("\nFeatures/Columns : \n", df_train.columns)
print("\n\nNumber of Features/Columns : ", len(df_train.columns))
print("\nNumber of Rows : ",len(df_train))
print("\n\nData Types :\n", df_train.dtypes)

print("\n Contains NaN/Empty cells : ", df_train.isnull().values.any())

print("\n Total empty cells by column :\n", df_train.isnull().sum(), "\n\n")

print("\n Perecentage of missing values in train datframe is: \n", str(100*(df_train.isnull().sum().sum()/len(df_train)))+'%')


EDA on Training Set

##############################

Features/Columns : 
 Index(['manufacturer_name', 'model_name', 'transmission', 'color',
       'odometer_value', 'year_produced', 'engine_fuel', 'engine_has_gas',
       'engine_type', 'engine_capacity', 'body_type', 'has_warranty', 'state',
       'drivetrain', 'feature_0', 'feature_1', 'feature_2', 'feature_3',
       'feature_4', 'feature_5', 'feature_6', 'feature_7', 'feature_8',
       'feature_9', 'duration_listed', 'price_usd'],
      dtype='object')


Number of Features/Columns :  26

Number of Rows :  30824


Data Types :
 manufacturer_name     object
model_name            object
transmission          object
color                 object
odometer_value         int64
year_produced          int64
engine_fuel           object
engine_has_gas         int64
engine_type           object
engine_capacity      float64
body_type             object
has_warranty           int64
state                 object
drivetrain            object
fe

In [48]:
# Importing the test dataset

df_test = pd.read_csv('Test.csv', engine= 'python', encoding= 'latin1')
df_test.drop('Unnamed: 0', axis = 1, inplace = True)
df_test.head()

,manufacturer_name,model_name,transmission,color,odometer_value,year_produced,engine_fuel,engine_has_gas,engine_type,engine_capacity,body_type,has_warranty,state,drivetrain,feature_0,feature_1,feature_2,feature_3,feature_4,feature_5,feature_6,feature_7,feature_8,feature_9,duration_listed
0,Renault,Megane,mechanical,blue,360000,1997,gasoline,0,gasoline,1.6,coupe,0,owned,front,0,1,0,0,0,0,0,1,0,1,114
1,Peugeot,206,mechanical,silver,267000,1999,gasoline,0,gasoline,1.4,hatchback,0,owned,front,0,0,0,0,0,0,0,0,0,1,67
2,Volkswagen,Sharan,mechanical,blue,172000,2000,gasoline,0,gasoline,2.0,minivan,0,owned,front,1,0,0,0,0,0,0,0,0,0,50
3,Volvo,XC60,mechanical,white,230000,2009,diesel,0,diesel,2.4,universal,0,owned,front,0,1,1,1,0,1,1,1,1,1,79
4,Mazda,3,mechanical,silver,206000,2007,gasoline,0,gasoline,1.6,sedan,0,owned,front,0,1,0,1,0,1,0,0,0,0,74


In [49]:
# priniting the details about the test dataset

print("#"*30)
print("\nEDA on Test Set\n")
print("#"*30)


print("\nFeatures/Columns : \n",df_test.columns)
print("\n\nNumber of Features/Columns : ",len(df_test.columns))
print("\nNumber of Rows : ",len(df_test))
print("\n\nData Types :\n", df_test.dtypes)
print("\n Contains NaN/Empty cells : ", df_test.isnull().values.any())
print("\n Total empty cells by column :\n", df_test.isnull().sum())
print("\n Perecentage of missing values in test datframe is: \n", str(100*(df_test.isnull().sum().sum()/len(df_test)))+'%')

##############################

EDA on Test Set

##############################

Features/Columns : 
 Index(['manufacturer_name', 'model_name', 'transmission', 'color',
       'odometer_value', 'year_produced', 'engine_fuel', 'engine_has_gas',
       'engine_type', 'engine_capacity', 'body_type', 'has_warranty', 'state',
       'drivetrain', 'feature_0', 'feature_1', 'feature_2', 'feature_3',
       'feature_4', 'feature_5', 'feature_6', 'feature_7', 'feature_8',
       'feature_9', 'duration_listed'],
      dtype='object')


Number of Features/Columns :  25

Number of Rows :  7707


Data Types :
 manufacturer_name     object
model_name            object
transmission          object
color                 object
odometer_value         int64
year_produced          int64
engine_fuel           object
engine_has_gas         int64
engine_type           object
engine_capacity      float64
body_type             object
has_warranty           int64
state                 object
drivetrain        

In [50]:
# Extract numerical columns from train dataframe
numerical_cols = [cname for cname in df_train.columns if
                df_train[cname].dtype in ['int64', 'float64']]

num_train= df_train[numerical_cols].copy()

num_train.head()

,odometer_value,year_produced,engine_has_gas,engine_capacity,has_warranty,feature_0,feature_1,feature_2,feature_3,feature_4,feature_5,feature_6,feature_7,feature_8,feature_9,duration_listed,price_usd
0,245000,2006,0,1.6,0,1,0,0,0,0,0,0,0,0,0,7,4250.0
1,265542,2002,0,3.3,0,0,1,0,0,0,0,0,0,0,0,133,4000.0
2,168000,2009,0,1.8,0,0,0,0,1,0,0,1,1,1,1,0,10900.0
3,225522,2008,0,1.8,0,0,1,1,0,0,1,0,0,1,1,20,6999.0
4,370000,1991,0,1.8,0,0,0,0,0,0,0,0,0,0,1,160,1600.0


In [51]:
# pRINT TEST SET

num_test = df_test[numerical_cols[:-1]]
num_test.head()

,odometer_value,year_produced,engine_has_gas,engine_capacity,has_warranty,feature_0,feature_1,feature_2,feature_3,feature_4,feature_5,feature_6,feature_7,feature_8,feature_9,duration_listed
0,360000,1997,0,1.6,0,0,1,0,0,0,0,0,1,0,1,114
1,267000,1999,0,1.4,0,0,0,0,0,0,0,0,0,0,1,67
2,172000,2000,0,2.0,0,1,0,0,0,0,0,0,0,0,0,50
3,230000,2009,0,2.4,0,0,1,1,1,0,1,1,1,1,1,79
4,206000,2007,0,1.6,0,0,1,0,1,0,1,0,0,0,0,74


In [52]:
# Extract categorical features from dataframe and establish train set and test set
category_cols = [col for col in df_train.columns if df_train[col].dtype == "object"]

# Frame that contains category columns and target variable
category_frame = category_cols.copy()
category_frame.append('price_usd')

# Create test and train dataframe
cat_df_train = df_train[category_cols]
cat_df_test = df_test[category_cols]

# Print dataframe
cat_df_train.head()

,manufacturer_name,model_name,transmission,color,engine_fuel,engine_type,body_type,state,drivetrain
0,Ford,Fusion,mechanical,blue,gasoline,gasoline,hatchback,owned,front
1,Dodge,Caravan,automatic,silver,gasoline,gasoline,minivan,owned,front
2,Ford,Galaxy,mechanical,blue,diesel,diesel,minivan,owned,front
3,Mazda,6,mechanical,other,gasoline,gasoline,universal,owned,front
4,Audi,80,mechanical,black,gasoline,gasoline,sedan,owned,front


In [53]:
# Let's print the test set
cat_df_test.head()

,manufacturer_name,model_name,transmission,color,engine_fuel,engine_type,body_type,state,drivetrain
0,Renault,Megane,mechanical,blue,gasoline,gasoline,coupe,owned,front
1,Peugeot,206,mechanical,silver,gasoline,gasoline,hatchback,owned,front
2,Volkswagen,Sharan,mechanical,blue,gasoline,gasoline,minivan,owned,front
3,Volvo,XC60,mechanical,white,diesel,diesel,universal,owned,front
4,Mazda,3,mechanical,silver,gasoline,gasoline,sedan,owned,front


In [54]:
# let's have a look at how many labels each variable has

for col in category_cols:
    print(col, ': ', len(cat_df_train[col].unique()), ' labels')

manufacturer_name :  55  labels
model_name :  1070  labels
transmission :  2  labels
color :  12  labels
engine_fuel :  6  labels
engine_type :  3  labels
body_type :  12  labels
state :  3  labels
drivetrain :  3  labels


In [55]:
# Let's print the shape of this train and test dataframe
print("The shape of the train set is: ", cat_df_train.shape)

print("The shape of the train set is: ", cat_df_test.shape)

The shape of the train set is:  (30824, 9)
The shape of the train set is:  (7707, 9)


In [56]:
# Select categorical columns with relatively low cardinality (convenient but arbitrary)
low_cardinality = [cname for cname in category_cols if
                    cat_df_train[cname].nunique() < 5 and
                    cat_df_train[cname].dtype == "object"]
low_cardinality

['transmission', 'engine_type', 'state', 'drivetrain']

In [57]:
# Select categorical columns with relatively low cardinality (convenient but arbitrary)
high_cardinality = [cname for cname in category_cols if
                    cat_df_train[cname].nunique() > 5 and
                    cat_df_train[cname].dtype == "object"]
high_cardinality

['manufacturer_name', 'model_name', 'color', 'engine_fuel', 'body_type']

### Method 1: Label encoding

In [58]:
# Columns that can be safely label encoded in validation set
good_label_cols = [col for col in category_cols if
                   set(cat_df_train[col]) == set(cat_df_test[col])]


# Problematic columns that will be dropped from the validation dataset
bad_label_cols = list(set(category_cols) - set(good_label_cols)) 

print('Categorical columns that will be label encoded :', good_label_cols)
print('\nCategorical columns that will be dropped from the dataset for label encoding:', bad_label_cols)

Categorical columns that will be label encoded : ['manufacturer_name', 'transmission', 'color', 'engine_type', 'body_type', 'state', 'drivetrain']

Categorical columns that will be dropped from the dataset for label encoding: ['model_name', 'engine_fuel']


In [59]:
# Drop bad labels from train and test dataframe
cat_df_train_le = cat_df_train.drop(bad_label_cols, axis= 1)
cat_df_test_le = cat_df_test.drop(bad_label_cols, axis= 1)

clean_cat_df_train.head()

,manufacturer_name,transmission,color,engine_type,body_type,state,drivetrain
0,Ford,mechanical,blue,gasoline,hatchback,owned,front
1,Dodge,automatic,silver,gasoline,minivan,owned,front
2,Ford,mechanical,blue,diesel,minivan,owned,front
3,Mazda,mechanical,other,gasoline,universal,owned,front
4,Audi,mechanical,black,gasoline,sedan,owned,front


In [60]:
# Target encode specific columns
target_cols = clean_cat_df_train.columns
from sklearn.preprocessing import LabelEncoder
le = LabelEncoder()
cat_df_train_le[target_cols]= cat_df_train_le[target_cols].apply(le.fit_transform)
cat_df_test_le[target_cols]= cat_df_test_le[target_cols].apply(le.fit_transform)


cat_df_train_le.head()

,manufacturer_name,transmission,color,engine_type,body_type,state,drivetrain
0,14,1,1,2,2,2,1
1,12,0,8,2,6,2,1
2,14,1,1,0,6,2,1
3,30,1,6,2,10,2,1
4,2,1,0,2,8,2,1


In [61]:
 # We collect all of these features (numerical and categorical) in a new dataframe 
df_train_le = num_train.join(cat_df_test_le)
df_test_le = num_test.join(cat_df_test_le)

df_train_le.to_csv('df_train_le.csv')
df_test_le.to_csv('df_test_le.csv')

df_train_le.head()

,odometer_value,year_produced,engine_has_gas,engine_capacity,has_warranty,feature_0,feature_1,feature_2,feature_3,feature_4,feature_5,feature_6,feature_7,feature_8,feature_9,duration_listed,price_usd,manufacturer_name,transmission,color,engine_type,body_type,state,drivetrain
0,245000,2006,0,1.6,0,1,0,0,0,0,0,0,0,0,0,7,4250.0,39.0,1.0,1.0,2.0,1.0,2.0,1.0
1,265542,2002,0,3.3,0,0,1,0,0,0,0,0,0,0,0,133,4000.0,36.0,1.0,8.0,2.0,2.0,2.0,1.0
2,168000,2009,0,1.8,0,0,0,0,1,0,0,1,1,1,1,0,10900.0,48.0,1.0,1.0,2.0,6.0,2.0,1.0
3,225522,2008,0,1.8,0,0,1,1,0,0,1,0,0,1,1,20,6999.0,49.0,1.0,10.0,0.0,10.0,2.0,1.0
4,370000,1991,0,1.8,0,0,0,0,0,0,0,0,0,0,1,160,1600.0,30.0,1.0,8.0,2.0,8.0,2.0,1.0


## Method 2: Selective encoding


Selectively one-hot encoding top X=10 or 20 or 50 etc. values in specific columns. For example, we observe that the column manufacturer_name has 55 distinct values hence we choose to encode the top 10 models as 1 and others as 0

In [62]:
# let's find the top 20 most frequent categories for the variable manufaturer_name
cat_df_train_se = cat_df_train.copy()
cat_df_test_se = cat_df_test.copy()

# Sort column
cat_df_train_se.manufacturer_name.value_counts().sort_values(ascending=False).head(5)

Volkswagen    3406
Opel          2193
BMW           2091
Ford          2080
Renault       1999
Name: manufacturer_name, dtype: int64

In [63]:
# let's make a list with the most frequent categories of the variable manufacturer_name

top_labels_manu_name = [y for y in cat_df_train_se.manufacturer_name.value_counts().sort_values(ascending=False).head(10).index]
top_labels_manu_name

['Volkswagen',
 'Opel',
 'BMW',
 'Ford',
 'Renault',
 'Audi',
 'Mercedes-Benz',
 'Peugeot',
 'Citroen',
 'Nissan']

In [64]:
# let's find the top 20 most frequent categories for the variable model_name

cat_df_train_se.model_name.value_counts().sort_values(ascending=False).head(5)

Passat    1153
Astra      595
Golf       577
A6         546
Mondeo     503
Name: model_name, dtype: int64

In [65]:
# let's make a list with the most frequent categories of the variable model_name

top_labels_mod_name = [y for y in cat_df_train_se.model_name.value_counts().sort_values(ascending=False).head(5).index]
top_labels_mod_name

['Passat', 'Astra', 'Golf', 'A6', 'Mondeo']

In [66]:
# let's make a list with the most frequent categories of the variable model_name

top_labels_color = [y for y in cat_df_train_se.color.value_counts().sort_values(ascending=False).head(5).index]
top_labels_color

['black', 'silver', 'blue', 'white', 'grey']

In [67]:
# get whole set of dummy variables, for all the categorical variables

def one_hot_encoding_top_x(df, variable, top_x_labels):
    # function to create the dummy variables for the most frequent labels
    # we can vary the number of most frequent labels that we encode
    
    for label in top_x_labels:
        df[variable+'_'+label] = np.where(df[variable]==label, 1.0, 0.0)

In [68]:
# encode manufaturer_name into the 10 most frequent categories
one_hot_encoding_top_x(cat_df_train_se, 'manufacturer_name', top_labels_manu_name)
one_hot_encoding_top_x(cat_df_test_se, 'manufacturer_name', top_labels_manu_name)
cat_df_train_se.head()

,manufacturer_name,model_name,transmission,color,engine_fuel,engine_type,body_type,state,drivetrain,manufacturer_name_Volkswagen,manufacturer_name_Opel,manufacturer_name_BMW,manufacturer_name_Ford,manufacturer_name_Renault,manufacturer_name_Audi,manufacturer_name_Mercedes-Benz,manufacturer_name_Peugeot,manufacturer_name_Citroen,manufacturer_name_Nissan
0,Ford,Fusion,mechanical,blue,gasoline,gasoline,hatchback,owned,front,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0
1,Dodge,Caravan,automatic,silver,gasoline,gasoline,minivan,owned,front,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,Ford,Galaxy,mechanical,blue,diesel,diesel,minivan,owned,front,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0
3,Mazda,6,mechanical,other,gasoline,gasoline,universal,owned,front,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,Audi,80,mechanical,black,gasoline,gasoline,sedan,owned,front,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0


In [69]:
# encode manufaturer_name into the 10 most frequent categories
one_hot_encoding_top_x(cat_df_train_se, 'model_name', top_labels_mod_name)
one_hot_encoding_top_x(cat_df_test_se, 'model_name', top_labels_mod_name)
cat_df_train_se.head()

,manufacturer_name,model_name,transmission,color,engine_fuel,engine_type,body_type,state,drivetrain,manufacturer_name_Volkswagen,manufacturer_name_Opel,manufacturer_name_BMW,manufacturer_name_Ford,manufacturer_name_Renault,manufacturer_name_Audi,manufacturer_name_Mercedes-Benz,manufacturer_name_Peugeot,manufacturer_name_Citroen,manufacturer_name_Nissan,model_name_Passat,model_name_Astra,model_name_Golf,model_name_A6,model_name_Mondeo
0,Ford,Fusion,mechanical,blue,gasoline,gasoline,hatchback,owned,front,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,Dodge,Caravan,automatic,silver,gasoline,gasoline,minivan,owned,front,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,Ford,Galaxy,mechanical,blue,diesel,diesel,minivan,owned,front,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,Mazda,6,mechanical,other,gasoline,gasoline,universal,owned,front,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,Audi,80,mechanical,black,gasoline,gasoline,sedan,owned,front,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [70]:
# encode manufaturer_name into the 10 most frequent categories
one_hot_encoding_top_x(cat_df_train_se, 'color', top_labels_color)
one_hot_encoding_top_x(cat_df_test_se, 'color', top_labels_color)
cat_df_train_se.head()

,manufacturer_name,model_name,transmission,color,engine_fuel,engine_type,body_type,state,drivetrain,manufacturer_name_Volkswagen,manufacturer_name_Opel,manufacturer_name_BMW,manufacturer_name_Ford,manufacturer_name_Renault,manufacturer_name_Audi,manufacturer_name_Mercedes-Benz,manufacturer_name_Peugeot,manufacturer_name_Citroen,manufacturer_name_Nissan,model_name_Passat,model_name_Astra,model_name_Golf,model_name_A6,model_name_Mondeo,color_black,color_silver,color_blue,color_white,color_grey
0,Ford,Fusion,mechanical,blue,gasoline,gasoline,hatchback,owned,front,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0
1,Dodge,Caravan,automatic,silver,gasoline,gasoline,minivan,owned,front,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0
2,Ford,Galaxy,mechanical,blue,diesel,diesel,minivan,owned,front,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0
3,Mazda,6,mechanical,other,gasoline,gasoline,universal,owned,front,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,Audi,80,mechanical,black,gasoline,gasoline,sedan,owned,front,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0


In [71]:
# Drop manufaturer_name column in train and test set
cat_df_train_se = cat_df_train_se.drop(['manufacturer_name', 'model_name','color'], axis = 1)
cat_df_test_se = cat_df_test_se.drop(['manufacturer_name','model_name','color'], axis = 1)

cat_df_train_se.head()

,transmission,engine_fuel,engine_type,body_type,state,drivetrain,manufacturer_name_Volkswagen,manufacturer_name_Opel,manufacturer_name_BMW,manufacturer_name_Ford,manufacturer_name_Renault,manufacturer_name_Audi,manufacturer_name_Mercedes-Benz,manufacturer_name_Peugeot,manufacturer_name_Citroen,manufacturer_name_Nissan,model_name_Passat,model_name_Astra,model_name_Golf,model_name_A6,model_name_Mondeo,color_black,color_silver,color_blue,color_white,color_grey
0,mechanical,gasoline,gasoline,hatchback,owned,front,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0
1,automatic,gasoline,gasoline,minivan,owned,front,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0
2,mechanical,diesel,diesel,minivan,owned,front,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0
3,mechanical,gasoline,gasoline,universal,owned,front,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,mechanical,gasoline,gasoline,sedan,owned,front,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0


In [72]:
# Target encode specific columns
target_cols = cat_df_train_se.columns
from sklearn.preprocessing import LabelEncoder
le2 = LabelEncoder()
cat_df_train_se[target_cols]= cat_df_train_se[target_cols].apply(le2.fit_transform)
cat_df_test_se[target_cols]= cat_df_test_se[target_cols].apply(le2.fit_transform)


cat_df_train_se.head()

,transmission,engine_fuel,engine_type,body_type,state,drivetrain,manufacturer_name_Volkswagen,manufacturer_name_Opel,manufacturer_name_BMW,manufacturer_name_Ford,manufacturer_name_Renault,manufacturer_name_Audi,manufacturer_name_Mercedes-Benz,manufacturer_name_Peugeot,manufacturer_name_Citroen,manufacturer_name_Nissan,model_name_Passat,model_name_Astra,model_name_Golf,model_name_A6,model_name_Mondeo,color_black,color_silver,color_blue,color_white,color_grey
0,1,3,2,2,2,1,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0
1,0,3,2,6,2,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0
2,1,0,0,6,2,1,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0
3,1,3,2,10,2,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
4,1,3,2,8,2,1,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,1,0,0,0,0


In [74]:
 # We collect all of these features (numerical and categorical) in a new dataframe 
df_train_se = num_train.join(cat_df_train_se)
df_test_se = num_test.join(cat_df_test_se)

df_train_se.to_csv('df_train_se.csv')
df_test_se.to_csv('df_test_se.csv')

df_train_se.head()
  


,odometer_value,year_produced,engine_has_gas,engine_capacity,has_warranty,feature_0,feature_1,feature_2,feature_3,feature_4,feature_5,feature_6,feature_7,feature_8,feature_9,duration_listed,price_usd,transmission,engine_fuel,engine_type,body_type,state,drivetrain,manufacturer_name_Volkswagen,manufacturer_name_Opel,manufacturer_name_BMW,manufacturer_name_Ford,manufacturer_name_Renault,manufacturer_name_Audi,manufacturer_name_Mercedes-Benz,manufacturer_name_Peugeot,manufacturer_name_Citroen,manufacturer_name_Nissan,model_name_Passat,model_name_Astra,model_name_Golf,model_name_A6,model_name_Mondeo,color_black,color_silver,color_blue,color_white,color_grey
0,245000,2006,0,1.6,0,1,0,0,0,0,0,0,0,0,0,7,4250.0,1,3,2,2,2,1,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0
1,265542,2002,0,3.3,0,0,1,0,0,0,0,0,0,0,0,133,4000.0,0,3,2,6,2,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0
2,168000,2009,0,1.8,0,0,0,0,1,0,0,1,1,1,1,0,10900.0,1,0,0,6,2,1,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0
3,225522,2008,0,1.8,0,0,1,1,0,0,1,0,0,1,1,20,6999.0,1,3,2,10,2,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
4,370000,1991,0,1.8,0,0,0,0,0,0,0,0,0,0,1,160,1600.0,1,3,2,8,2,1,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,1,0,0,0,0


In [75]:
print(df_train_se.isnull().sum().sum())
print(df_test_se.isnull().sum().sum())

0
0


### Method 4 Selective encoding

In [76]:
# Target encoding for manufature name and model name columns
cat_df_train_se2 = cat_df_train.copy()
cat_df_test_se2 = cat_df_test.copy()

cat_df_train_se2.head()

,manufacturer_name,model_name,transmission,color,engine_fuel,engine_type,body_type,state,drivetrain
0,Ford,Fusion,mechanical,blue,gasoline,gasoline,hatchback,owned,front
1,Dodge,Caravan,automatic,silver,gasoline,gasoline,minivan,owned,front
2,Ford,Galaxy,mechanical,blue,diesel,diesel,minivan,owned,front
3,Mazda,6,mechanical,other,gasoline,gasoline,universal,owned,front
4,Audi,80,mechanical,black,gasoline,gasoline,sedan,owned,front


In [77]:
# Apply traget encoding 
cols_target = ['manufacturer_name','model_name']
target_enc3 = ce.TargetEncoder(cols=cols_target)
target_enc3.fit(cat_df_train_se2[cols_target],num_train['price_usd'] )

# Transform the features, rename the columns with _target suffix, and join to dataframe
cat_df_train_se2[cols_target] = target_enc3.transform(cat_df_train_se2[cols_target]).add_suffix('_te')
cat_df_test_se2[cols_target] = target_enc3.transform(cat_df_test_se2[cols_target]).add_suffix('_te')

cat_df_train_se2.head()

,manufacturer_name,model_name,transmission,color,engine_fuel,engine_type,body_type,state,drivetrain
0,5029.863851,9575.330756,mechanical,blue,gasoline,gasoline,hatchback,owned,front
1,5735.589271,3179.271449,automatic,silver,gasoline,gasoline,minivan,owned,front
2,5029.863851,5543.469085,mechanical,blue,diesel,diesel,minivan,owned,front
3,4735.107530,8128.120448,mechanical,other,gasoline,gasoline,universal,owned,front
4,7121.501476,1643.286410,mechanical,black,gasoline,gasoline,sedan,owned,front


In [78]:
# Target encode specific columns
target_cols = cat_df_train_se2.columns
from sklearn.preprocessing import LabelEncoder
le3 = LabelEncoder()
cat_df_train_se2[target_cols]= cat_df_train_se2[target_cols].apply(le3.fit_transform)
cat_df_test_se2[target_cols]= cat_df_test_se2[target_cols].apply(le3.fit_transform)


cat_df_train_se2.head()

,manufacturer_name,model_name,transmission,color,engine_fuel,engine_type,body_type,state,drivetrain
0,20,612,1,1,3,2,2,2,1
1,24,226,0,8,3,2,6,2,1
2,20,395,1,1,0,0,6,2,1
3,18,538,1,6,3,2,10,2,1
4,29,101,1,0,3,2,8,2,1


In [79]:
 # We collect all of these features (numerical and categorical) in a new dataframe 
df_train_se2 = num_train.join(cat_df_train_se2)
df_test_se2 = num_test.join(cat_df_test_se2)

df_train_se2.to_csv('df_train_se2.csv')
df_test_se2.to_csv('df_test_se2.csv')

df_train_se2.head()

,odometer_value,year_produced,engine_has_gas,engine_capacity,has_warranty,feature_0,feature_1,feature_2,feature_3,feature_4,feature_5,feature_6,feature_7,feature_8,feature_9,duration_listed,price_usd,manufacturer_name,model_name,transmission,color,engine_fuel,engine_type,body_type,state,drivetrain
0,245000,2006,0,1.6,0,1,0,0,0,0,0,0,0,0,0,7,4250.0,20,612,1,1,3,2,2,2,1
1,265542,2002,0,3.3,0,0,1,0,0,0,0,0,0,0,0,133,4000.0,24,226,0,8,3,2,6,2,1
2,168000,2009,0,1.8,0,0,0,0,1,0,0,1,1,1,1,0,10900.0,20,395,1,1,0,0,6,2,1
3,225522,2008,0,1.8,0,0,1,1,0,0,1,0,0,1,1,20,6999.0,18,538,1,6,3,2,10,2,1
4,370000,1991,0,1.8,0,0,0,0,0,0,0,0,0,0,1,160,1600.0,29,101,1,0,3,2,8,2,1


In [80]:
print(df_train_se2.isnull().sum().sum())
print(df_test_se2.isnull().sum().sum())

0
0
